In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.nn.utils import clip_grad_norm_
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm


# compute metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import metrics


In [2]:
tax=pd.read_csv("/home/azureuser/cloudfiles/code/Users/rishabh.chanana.adm/AutoTagging/Active Data Sources/x_snc_taxonomy_sys_map_new.csv")
end_val=tax['ending_node'].to_list()
path=tax['path'].to_list()
path=list(map(lambda st:st.split('.'), path))
tax_path=dict(zip(end_val, path))

In [3]:
kb_data=pd.read_parquet('/home/azureuser/cloudfiles/code/Users/rishabh.chanana.adm/AutoTagging/Active Data Sources/public_customer_kbs.parquet', engine='pyarrow')

In [4]:
kb_data = kb_data.rename(columns={'NUMBER': 'number', 'SHORT_DESCRIPTION': 'desc', 'TEXT':'text'})

In [5]:
kb_data=kb_data[['number','desc','text']]

In [6]:
# def read_all_child_tables_for_kb_knowledge():

#     knowledge = pd.read_csv("/home/azureuser/cloudfiles/code/Users/rishabh.chanana.adm/AutoTagging/Active Data Sources/5000internal/csvs/kb_knowledge.csv")
#     knowledge['kb_desc'] = knowledge['text'].apply(utils.pii_scrubber)
#     knowledge['kb_desc'] = knowledge['kb_desc'].apply(utils.html_cleanup)
#     knowledge['kb_desc'] = knowledge['kb_desc'].apply(utils.scrub_words)

#     to_for_cs = pd.read_csv("/home/azureuser/cloudfiles/code/Users/rishabh.chanana.adm/AutoTagging/Active Data Sources/5000internal/csvs/u_kb_template_kcs_how_to_for_cs.csv")
#     to_for_cs=to_for_cs.fillna(' ')
#     cols=list(to_for_cs)
#     cols.remove('number')
#     to_for_cs['combined']=to_for_cs[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
#     to_for_cs['kb_desc'] = to_for_cs['combined'].apply(utils.html_cleanup)
#     to_for_cs['kb_desc'] = to_for_cs['kb_desc'].apply(utils.scrub_words)
#     to_for_cs['kb_desc'] = to_for_cs['kb_desc'].apply(utils.pii_scrubber)



#     solution_for_cs = pd.read_csv("/home/azureuser/cloudfiles/code/Users/rishabh.chanana.adm/AutoTagging/Active Data Sources/5000internal/csvs/u_kb_template_kcs_solution_for_cs.csv")
#     solution_for_cs=solution_for_cs.fillna(' ')
#     cols=list(solution_for_cs)
#     cols.remove('number')
#     solution_for_cs['combined']=solution_for_cs[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
#     solution_for_cs['kb_desc'] = solution_for_cs['combined'].apply(utils.html_cleanup)
#     solution_for_cs['kb_desc'] = solution_for_cs['kb_desc'].apply(utils.scrub_words)
#     solution_for_cs['kb_desc'] = solution_for_cs['kb_desc'].apply(utils.pii_scrubber)



#     what_is_for_cs = pd.read_csv("/home/azureuser/cloudfiles/code/Users/rishabh.chanana.adm/AutoTagging/Active Data Sources/5000internal/csvs/u_kb_template_kcs_what_is_for_cs.csv")
#     what_is_for_cs=what_is_for_cs.fillna(' ')
#     cols=list(what_is_for_cs)
#     cols.remove('number')
#     what_is_for_cs['combined']=what_is_for_cs[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
#     what_is_for_cs['kb_desc'] = what_is_for_cs['combined'].apply(utils.html_cleanup)
#     what_is_for_cs['kb_desc'] = what_is_for_cs['kb_desc'].apply(utils.scrub_words)
#     what_is_for_cs['kb_desc'] = what_is_for_cs['kb_desc'].apply(utils.pii_scrubber)

   

#     return(to_for_cs, solution_for_cs, what_is_for_cs,knowledge)
# to_for_cs, solution_for_cs, what_is_for_cs, knowledge = read_all_child_tables_for_kb_knowledge()


In [7]:
# to_for_cs=to_for_cs[['number','kb_desc']]
# solution_for_cs=solution_for_cs[['number','kb_desc']]
# what_is_for_cs=what_is_for_cs[['number','kb_desc']]
# knowledge=knowledge[['number','kb_desc']]
# combined=pd.concat([to_for_cs,solution_for_cs,what_is_for_cs,knowledge], ignore_index=True)
# multi_path=pd.read_csv("/home/azureuser/cloudfiles/code/Users/rishabh.chanana.adm/AutoTagging/Active Data Sources/5000internal/kb_5000_multi_path.csv",usecols = ['number','testfield','meta_description','short_description'])
# no_path=pd.read_csv("/home/azureuser/cloudfiles/code/Users/rishabh.chanana.adm/AutoTagging/Active Data Sources/5000internal/kb_5000_no_path.csv",usecols = ['number','meta_description','short_description'])
# no_product=pd.read_csv("/home/azureuser/cloudfiles/code/Users/rishabh.chanana.adm/AutoTagging/Active Data Sources/5000internal/kb_5000_noproduct.csv",usecols = ['number','meta_description','short_description'])
# no_path=pd.concat([no_path,no_product], ignore_index=True)
# multi_path=multi_path.merge(combined, on='number', how='left')
# no_path=no_path.merge(combined, on='number', how='left')
# multi_path[multi_path['kb_desc']=='nan']




In [8]:
allkbmeta1=pd.read_csv("/home/azureuser/cloudfiles/code/Users/rishabh.chanana.adm/AutoTagging/Active Data Sources/TaaS/allkbs_meta1.csv")
allkbmeta2=pd.read_csv("/home/azureuser/cloudfiles/code/Users/rishabh.chanana.adm/AutoTagging/Active Data Sources/TaaS/allkbs_meta2.csv")
allkbmeta= pd.concat([allkbmeta1,allkbmeta2])
taxresults=pd.read_csv("/home/azureuser/cloudfiles/code/Users/rishabh.chanana.adm/AutoTagging/Active Data Sources/allresults.csv")


In [9]:
allkbmeta

,number,kb_knowledge_base,short_description,meta_description,category,kb_category
0,KB0984381,Product Documentation: Utah,Populate the stakeholder registry,Previous topic | Next topic | IT Business Mana...,NaN,IT Service Management
1,KB1124068,Product Documentation: Tokyo,SAP Successfactors to ServiceNow field mapping,The Trigger SAP Successfactors Integration flo...,NaN,Employee Service Management
2,KB1124241,Product Documentation: Tokyo,SAP Successfactors: Known Issues and Limitations,The ServiceNow® HR Service Delivery integratio...,NaN,Employee Service Management
3,KB1125183,Product Documentation: Tokyo,IT Service Management,The IT Service Management (ITSM) solution prov...,NaN,IT Service Management
4,KB1125184,Product Documentation: Tokyo,Script includes and customization,Many Script Includes are provided by default w...,NaN,IT Service Management
...,...,...,...,...,...,...
73413,KBB0010043,"Trust, Privacy, and Compliance",TPC_LINKS_PUBLIC_Microsoft Partnership,Find out more about ServiceNow's partnership w...,NaN,Miscellaneous
73414,KBB0010048,"Trust, Privacy, and Compliance",TPC_LINKS_PUBLIC_PRODUCT Business Continuity,Find out more about ServiceNow's Business Cont...,NaN,Miscellaneous
73415,KBB0010057,"Trust, Privacy, and Compliance",TPC_HEADER_PUBLIC_More information,More information,NaN,Miscellaneous
73416,KBB0010059,"Trust, Privacy, and Compliance",TPC_HEADER_CUSTOMER_More information,NaN,NaN,Miscellaneous


In [10]:
df2=pd.read_pickle('/home/azureuser/cloudfiles/code/Users/rishabh.chanana.adm/AutoTagging/Active Data Sources/TaaS/taas_trainingdata500_3.pkl')
df=pd.read_pickle('/home/azureuser/cloudfiles/code/Users/rishabh.chanana.adm/AutoTagging/Active Data Sources/TaaS/taas_trainingdata100.pkl')

In [11]:
df=df.reset_index(drop=True)
df = df.rename(columns={'label': 'og_label'})

In [12]:

df2=df2.reset_index(drop=True)
df2 = df2.rename(columns={'label': 'og_label'})

In [13]:
# old_labels=set(df.label.to_list())
# num=0
# new_labels_dict={}
# for i in old_labels:
#     new_labels_dict[i]=num
#     num+=1
# print(new_labels_dict)

In [14]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['label'] = labelencoder.fit_transform(df['og_label'])


In [15]:
df2['label'] = labelencoder.fit_transform(df2['og_label'])


In [14]:
# def assigntrainnums(val):
#     return(new_labels_dict[val])

In [19]:
len(set(df.label))

244

In [15]:
# df['label']=df['og_label'].apply(assigntrainnums)
# df

In [21]:
label_dict_100 = dict(zip(df.label,df.taxonomy))
label_dict_500 = dict(zip(df2.label,df2.taxonomy))

In [26]:
import pickle
with open('label_dict_100.pickle', 'wb') as f1:
    pickle.dump(label_dict_100, f1)
with open('label_dict_500.pickle', 'wb') as f2:
    pickle.dump(label_dict_500, f2)

In [32]:
len(label_dict_500)

52

In [25]:
label_dict_500

{0: "['Employee Service Management', 'Employee Center', 'Content Publishing']",
 39: "['Telecommunications Service Management', 'Order Management for Telecommunications, Media, and Technology']",
 29: "['Security Operations (Product Line)', 'Security Operations']",
 50: "['Now Platform', 'Configuration Management Database (CMDB)']",
 48: "['Security Operations (Product Line)', 'Security Incident Response', 'Security Incident Response integrations']",
 40: "['IT Operations Management', 'ITOM Visibility']",
 20: "['Customer Service Management', 'Customer Operations for Customer Service Management', 'Data Management for Customer Service']",
 7: "['Governance, Risk, and Compliance', 'Policy and Compliance Management']",
 24: "['App Engine', 'System Web Services', 'REST APIs', 'Contact API']",
 13: "['IT Operations Management', 'ITOM Visibility', 'Discovery']",
 49: "['Now Platform', 'Automated Test Framework (ATF)']",
 37: "['IT Asset Management', 'Software Asset Management']",
 17: "['Now

In [24]:
len(label_dict_100)

244

In [16]:
df

,number,taxonomy,taxonomy_nums,og_label,text,label
0,KB1473471,"['IT Asset Management', 'Enterprise Asset Mana...",859,859,resolve and close an enterprise asset manageme...,77
1,KB1142273,"['IT Operations Management', 'ITOM Visibility'...",1345,-1,examples of discovery behavior functionalities...,0
2,KB1154025,"['IT Asset Management', 'Cloud Cost Management...",70,-1,configure reservation plans operations configu...,0
3,KB1448456,"['Governance, Risk, and Compliance', 'Business...",84,-1,impact rating record form impact rating record...,0
4,KB1244838,"['IT Asset Management', 'Enterprise Asset Mana...",859,859,close an enterprise asset reclamation request ...,77
...,...,...,...,...,...,...
24395,KB1132016,"['Telecommunications Service Management', 'Tel...",304,304,creating equipment relationships with inventor...,27
24396,KB1135654,"['Employee Service Management', 'Workplace Ser...",1625,1625,analytics and reporting solutions for workplac...,145
24397,KB1487235,"['IT Service Management', 'Workforce Optimizat...",2448,2448,setting up channels in workforce optimization ...,219
24398,KB1140835,"['Governance, Risk, and Compliance', 'Risk Man...",2053,2053,entity tiers in grc entity tiers in grc by cre...,187


In [17]:
len(df.label)

24400

In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# tokenizer = BertTokenizer.from_pretrained('FacebookAI/roberta-base', do_lower_case=True)

In [19]:
# from transformers import AutoModel,AutoTokenizer
# name = "FacebookAI/roberta-base"
# tokenizer = AutoTokenizer.from_pretrained(name, add_prefix_space=True, use_fast=False)
# model = BertForSequenceClassification.from_pretrained(name)


In [20]:
train_df, val_df = train_test_split(df, test_size=0.30, stratify=df['label'], random_state=42)
test_df, val_df = train_test_split(val_df, test_size=0.50, stratify=val_df['label'], random_state=42)


In [21]:
def encode(docs):
    '''
    This function takes list of texts and returns input_ids and attention_mask of texts
    '''
    encoded_dict = tokenizer.batch_encode_plus(docs, add_special_tokens=True, max_length=512, padding='max_length',
                            return_attention_mask=True, truncation=True, return_tensors='pt')
    input_ids = encoded_dict['input_ids']
    attention_masks = encoded_dict['attention_mask']
    return input_ids, attention_masks

In [22]:
train_input_ids, train_att_masks = encode(train_df['text'].values.tolist())
valid_input_ids, valid_att_masks = encode(val_df['text'].values.tolist())
test_input_ids, test_att_masks = encode(test_df['text'].values.tolist())

In [23]:
train_y = torch.LongTensor(train_df['label'].values.tolist())
valid_y = torch.LongTensor(val_df['label'].values.tolist())
test_y = torch.LongTensor(test_df['label'].values.tolist())
train_y.size(),valid_y.size(),test_y.size()

(torch.Size([17080]), torch.Size([3660]), torch.Size([3660]))

In [24]:
BATCH_SIZE = 32
train_dataset = TensorDataset(train_input_ids, train_att_masks, train_y)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

valid_dataset = TensorDataset(valid_input_ids, valid_att_masks, valid_y)
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_input_ids, test_att_masks, test_y)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [25]:
# model = AutoModel.from_pretrained(name, num_labels=N_labels,
#                                                       output_attentions=False,
#                                                       output_hidden_states=False)


In [26]:
N_labels = len(train_df.label.unique())
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels=N_labels,
                                                      output_attentions=False,
                                                      output_hidden_states=False)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [28]:
# import gc

# model.cpu()
# del model
# gc.collect()
# torch.cuda.empty_cache()

In [29]:
model = model.to(device)

In [30]:
EPOCHS = 100
LEARNING_RATE = 2e-6

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, 
             num_warmup_steps=0,
            num_training_steps=len(train_dataloader)*EPOCHS )

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [31]:
import math

train_loss_per_epoch = []
val_loss_per_epoch = []
val_loss_max=float('inf')

for epoch_num in range(EPOCHS):
    print('Epoch: ', epoch_num + 1)
    '''
    Training
    '''
    model.train()
    train_loss = 0
    for step_num, batch_data in enumerate(tqdm(train_dataloader,desc='Training')):
        input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)
        
        loss = output.loss
        train_loss += loss.item()

        model.zero_grad()
        loss.backward()
        del loss

        clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    train_loss_per_epoch.append(train_loss / (step_num + 1))              


    '''
    Validation
    '''
    model.eval()
    valid_loss = 0
    valid_pred = []
    with torch.no_grad():
        for step_num_e, batch_data in enumerate(tqdm(valid_dataloader,desc='Validation')):
            input_ids, att_mask, labels = [data.to(device) for data in batch_data]
            output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

            loss = output.loss
            valid_loss += loss.item()
   
            valid_pred.append(np.argmax(output.logits.cpu().detach().numpy(),axis=-1))
        
    val_loss_per_epoch.append(valid_loss / (step_num_e + 1))
    valid_pred = np.concatenate(valid_pred)
    if(valid_loss<val_loss_max):
        valid_loss_max=valid_loss
        torch.save(model.state_dict(), 'model.pk')
    else:
        break

    '''
    Loss message
    '''
    print("{0}/{1} train loss: {2} ".format(step_num+1, math.ceil(len(train_df) / BATCH_SIZE), train_loss / (step_num + 1)))
    print("{0}/{1} val loss: {2} ".format(step_num_e+1, math.ceil(len(val_df) / BATCH_SIZE), valid_loss / (step_num_e + 1)))
    print("Accuracy=",metrics.accuracy_score(val_df['label'].to_numpy(),valid_pred)
)

Epoch:  1


Training:   0%|          | 0/534 [00:00<?, ?it/s]

Validation:   0%|          | 0/115 [00:00<?, ?it/s]

534/534 train loss: 5.493703666251251 
115/115 val loss: 5.430581283569336 
Accuracy= 0.006010928961748634
Epoch:  2


Training:   0%|          | 0/534 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [41]:
kb_data

,number,desc,text
0,KB0521688,Troubleshooting Network Performance_Data Colle...,servicenow works to build and maintain an indu...
1,KB0535320,Troubleshooting reports that take a long time ...,troubleshooting reports that take a long time ...
2,KB0535508,Change Requests can be saved with a start date...,description when creating or modifying a chang...
3,KB0538138,Troubleshooting issues with email attachments,this article guides you through the process of...
4,KB0538170,User domain does not match the workflow domain,user domain does not match the workflow domain...
...,...,...,...
156764,KB1633216,RESTAPIRequest - queryString,restapirequest restapirequest the restapireque...
156765,KB1633268,Configure outbound configurations in ServiceNo...,set up slack spoke set up slack spoke integrat...
156766,KB1633347,Install Windows App activity,install windows app activity install windows a...
156767,KB1633370,RESTAPIRequest - pathParams,restapirequest restapirequest the restapireque...


In [42]:
model.eval()
test_pred = []
test_loss= 0
with torch.no_grad():
    for step_num, batch_data in tqdm(enumerate(test_dataloader)):
        input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

        loss = output.loss
        test_loss += loss.item()
   
        test_pred.append(np.argmax(output.logits.cpu().detach().numpy(),axis=-1))
test_pred = np.concatenate(test_pred)

0it [00:00, ?it/s]

In [43]:
print("Accuracy=",metrics.accuracy_score(test_df['label'].to_numpy(),test_pred))


Accuracy= 0.7475409836065574


In [44]:
print('classifiation report')
print(classification_report(test_pred, test_df['label'].to_numpy()))

classifiation report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.93      0.82      0.87        17
           2       0.67      0.62      0.65        16
           3       1.00      0.94      0.97        16
           4       0.87      0.81      0.84        16
           5       0.47      0.78      0.58         9
           6       0.87      0.87      0.87        15
           7       0.80      0.63      0.71        19
           8       0.67      0.62      0.65        16
           9       0.73      0.50      0.59        22
          10       0.40      0.40      0.40        15
          11       1.00      0.58      0.73        26
          12       0.87      0.87      0.87        15
          13       0.93      0.93      0.93        15
          14       0.53      0.73      0.62        11
          15       0.80      0.80      0.80        15
          16       0.07      0.50      0.12         2
      

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
new_labels_dict

NameError: name 'new_labels_dict' is not defined